# Evaluation LDA


In [2]:
# imports
import pandas as pd
import numpy as np
import itertools
from itertools import permutations
from sklearn import metrics

In [ ]:
path = '/home/ubuntu/ullrich/data/'

In [3]:
#read csv file from predicted topics
df = pd.read_csv(path + 'predicted_LDA_4classes.csv', sep=',')

In [4]:
# drop unnessecary columns 
df = df.drop(['Unnamed: 0'], axis=1)
# replace topic name
df = df.replace({'class':'ErnÃ¤hrung'},{'class':'Ernährung'})

In [5]:
df

dbrecordid  \
0                                    AGRICOLAFNI87002336   
1                                              M30394453   
2      BASE::ftdoajarticles:oai:doaj.org/article:b3da...   
3                                              M29238402   
4      BASE::ftdoajarticles:oai:doaj.org/article:1775...   
...                                                  ...   
96731                                AGRISUS201600128370   
96732      BASE::ftagecon:oai:ageconsearch.umn.edu:18859   
96733                                AGRICOLAIND20553515   
96734                                          M34357901   
96735                                          M34721997   

                      class  topic  certainty  
0                   Medizin    3.0   0.620343  
1                   Medizin    2.0   0.610879  
2                   Medizin    0.0   0.941765  
3            Landwirtschaft    1.0   0.634890  
4      Umweltwissenschaften    0.0   0.967090  
...                     ...    ...        ...  
96731        Landwirtschaft    3.0   0.689258  
96732  Umweltwissenschaften    NaN        NaN  
96733        Landwirtschaft    NaN        NaN  
96734  Umweltwissenschaften    2.0   0.883650  
96735        Landwirtschaft    2.0   0.626722  

[96736 rows x 4 columns]

In [8]:
#define all possible combinations of the classes
classes = ['Medizin', 'Landwirtschaft', 'Umweltwissenschaften', 'Ernährung']
topics = [0.0,1.0,2.0,3.0]

unique_combinations = []
permut = itertools.permutations(classes, len(topics))

for comb in permut:
    zipped = zip(comb, topics)
    unique_combinations.append(list(zipped))
 
# printing unique_combination list
print(unique_combinations)

[[('Medizin', 0.0), ('Landwirtschaft', 1.0), ('Umweltwissenschaften', 2.0), ('Ernährung', 3.0)], [('Medizin', 0.0), ('Landwirtschaft', 1.0), ('Ernährung', 2.0), ('Umweltwissenschaften', 3.0)], [('Medizin', 0.0), ('Umweltwissenschaften', 1.0), ('Landwirtschaft', 2.0), ('Ernährung', 3.0)], [('Medizin', 0.0), ('Umweltwissenschaften', 1.0), ('Ernährung', 2.0), ('Landwirtschaft', 3.0)], [('Medizin', 0.0), ('Ernährung', 1.0), ('Landwirtschaft', 2.0), ('Umweltwissenschaften', 3.0)], [('Medizin', 0.0), ('Ernährung', 1.0), ('Umweltwissenschaften', 2.0), ('Landwirtschaft', 3.0)], [('Landwirtschaft', 0.0), ('Medizin', 1.0), ('Umweltwissenschaften', 2.0), ('Ernährung', 3.0)], [('Landwirtschaft', 0.0), ('Medizin', 1.0), ('Ernährung', 2.0), ('Umweltwissenschaften', 3.0)], [('Landwirtschaft', 0.0), ('Umweltwissenschaften', 1.0), ('Medizin', 2.0), ('Ernährung', 3.0)], [('Landwirtschaft', 0.0), ('Umweltwissenschaften', 1.0), ('Ernährung', 2.0), ('Medizin', 3.0)], [('Landwirtschaft', 0.0), ('Ernährung',

In [9]:
#get lists of pred and true values
pred_test = df['topic'].values.tolist()
true_test = df['class'].values.tolist()

In [10]:
#replace items in predicted list
def replace_items(pred_list, true_list):
    liste = list(pred_list)
    for i in range(len(liste)):
        for tupel in true_list:
            if liste[i] == tupel[1]:
                liste[i] = tupel[0]
    return liste

In [11]:
#create f1-score for every combination
def f1_score(combinations, pred_list, true_list):
    result = []
    for combi in combinations:
        res = {}
        pred = replace_items(pred_list,combi)
        f1 = metrics.f1_score(true_list, pred, average='weighted')
        res.update({'Combi': combi,'f1_score': f1})
        result.append(res)
    return result

In [12]:
#get dictionary for each combination and every score
res_dict = f1_score(unique_combinations, pred_test, true_test)

In [13]:
#print all f1-scores in the result dict
[x['f1_score'] for x in res_dict]

[0.2674848618667972,
 0.2347207129749406,
 0.234946932649612,
 0.2260381336831592,
 0.22606719774999584,
 0.24992254767539965,
 0.2311360248579004,
 0.19837187596604378,
 0.19266108274677343,
 0.18512267577386618,
 0.18378134784715727,
 0.20900708976610663,
 0.20081049858968458,
 0.19190169962323175,
 0.19487348569574278,
 0.1873350787228355,
 0.17731117150434525,
 0.17868156349789074,
 0.2077583607434947,
 0.2316137106688986,
 0.20182134784955294,
 0.22704708976850232,
 0.19313876855777157,
 0.1945091605513171]

In [20]:
#get highest value of dictionary
def highest_val(lst, key, key2):
    highest_value = None
    for dict in lst:
        score = dict[key]
        combi = dict[key2]  
        if highest_value is None or score > highest_value:
            highest_value = score
            pred_combi = combi
    return highest_value, pred_combi

In [21]:
#get the highest values with combination
f1 = highest_val(res_dict,'f1_score','Combi')

In [22]:
f1

(0.2674848618667972,
 [('Medizin', 0.0),
  ('Landwirtschaft', 1.0),
  ('Umweltwissenschaften', 2.0),
  ('Ernährung', 3.0)])